# Loading Data

In [2]:
##required packages:
import pandas as pd
import openpyxl # for .xlsx files
import json
import uuid

In [3]:
df = pd.read_excel(r"C:\Users\VivoBook\OneDrive\Desktop\PandasProject\raw_data.xlsx")

### dim_comm_type table

In [4]:
df_comm_type = pd.DataFrame() #Empty DataFrame for comm_type
df_comm_type['comm_type'] = df['comm_type'].drop_duplicates()
df_comm_type.index += 1 #Later I will use index for comm_type_id so, it should start from 1.

In [5]:
df_comm_type['comm_type_id'] = df_comm_type.index
df_comm_type

,comm_type,comm_type_id
1,call,1
3,meeting,3


In [6]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl') as writer:
    df_comm_type.to_excel(writer,sheet_name='dim_comm_type',index=False)

### Subject table

In [7]:
df_subject = pd.DataFrame() #Empty DataFrame for subject
df_subject['subject'] = df['subject'].drop_duplicates()
df_subject.index += 1 
df_subject['subject_id'] = df_subject.index

In [8]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_subject.to_excel(writer,sheet_name='dim_subject',index=False)

### Now, we can work with raw_content, first we need to convert string to json format

In [9]:
def extract_all_json_objects(text):
    objects = []
    decoder = json.JSONDecoder()
    idx = 0
    while idx < len(text):
        try:
            obj, next_idx = decoder.raw_decode(text, idx)
            objects.append(obj)
            idx = next_idx
        except json.JSONDecodeError:
            break
    return objects

### dim_calendar table

In [10]:
df_calendar = pd.DataFrame() #Empty DataFrame for calendar
df_calendar['raw_calendar_id'] = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('calendar_id') if extract_all_json_objects(x) else None
).drop_duplicates()
df_calendar.index += 1 
df_calendar['calendar_id'] = df_calendar.index

In [11]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_calendar.to_excel(writer,sheet_name='dim_calendar',index=False)

### dim_audio

In [12]:
df_audio = pd.DataFrame() 
df_audio['raw_audio_url'] = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('audio_url') if extract_all_json_objects(x) else None
).drop_duplicates()
df_audio.index += 1 
df_audio['audio_id'] = df_audio.index

In [13]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_audio.to_excel(writer,sheet_name='dim_audio',index=False)

### dim_video

In [14]:
df_video = pd.DataFrame() 
df_video['raw_video_url'] = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('video_url') if extract_all_json_objects(x) else None
).drop_duplicates()
df_video.dropna(inplace=True)
df_video.index += 1 
df_video['video_id'] = df_video.index

In [15]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_video.to_excel(writer,sheet_name='dim_video',index=False)

### dim_transcript

In [16]:
df_transcript = pd.DataFrame() 
df_transcript['raw_transcript_url'] = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('transcript_url') if extract_all_json_objects(x) else None
).drop_duplicates()
df_transcript.dropna(inplace=True)
df_transcript.index += 1 
df_transcript['transcript_id'] = df_transcript.index

In [17]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_transcript.to_excel(writer,sheet_name='dim_transcript',index=False)

### dim_user

In [18]:
list_attendees = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('meeting_attendees') if extract_all_json_objects(x) else None
)
list_speakers = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('speakers') if extract_all_json_objects(x) else None
)
list_host_email = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('host_email') if extract_all_json_objects(x) else None
)
list_participants = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('participants') if extract_all_json_objects(x) else None
)
list_organizer_email = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('organizer_email') if extract_all_json_objects(x) else None
)
list_meeting_id = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('id') if extract_all_json_objects(x) else None
)
list_comm_id = df['id']

In [19]:
rows = []

for i in range(len(list_comm_id)):
    attendees_list = list_attendees[i] or []
    speakers_list = list_speakers[i] or []
    host = list_host_email[i]
    participants_list = list_participants[i] or []
    organiser = list_organizer_email[i] 
    comm_id = list_comm_id[i]   

    users_emails = set()
    added_names = set()

    for participant_email in participants_list:
        
        participant_email = participant_email.lower()
        matched = False
        
        for speaker in speakers_list:
            
            speaker_name = speaker.get('name', '').strip().lower()
            speaker_parts = speaker_name.split()

            first_name = speaker_parts[0] if speaker_parts else ''
            last_name = speaker_parts[-1] if len(speaker_parts) > 1 else ''
            
            patterns = [
                first_name,
                last_name,
                f"{first_name}.{last_name}",
                f"{first_name}_{last_name}",
                f"{last_name}.{first_name}",
                f"{first_name}{last_name}",
                f"{last_name}{first_name}"
            ]
            
            if any(p in participant_email for p in patterns if p):
                rows.append({
                    'user_id': str(uuid.uuid5(uuid.NAMESPACE_DNS, participant_email)),
                    'name': speaker.get('name'),
                    'email': participant_email,
                    'location': None,
                    'displayName': None,
                    'phoneNumber': None,
                    'comm_id': comm_id
                })
                matched = True
                added_names.add(speaker_name)
                break
        
        if not matched:
            rows.append({
                'user_id': str(uuid.uuid5(uuid.NAMESPACE_DNS, participant_email)),
                'name': None,
                'email': participant_email,
                'location': None,
                'displayName': None,
                'phoneNumber': None,
                'comm_id': comm_id
            })
        
        users_emails.add(participant_email)

    for attendee in attendees_list:
        
        email = attendee.get('email', '').lower()
        
        if email and email not in users_emails:
            rows.append({
                'user_id': str(uuid.uuid5(uuid.NAMESPACE_DNS, email)),
                'name': attendee.get('name'),
                'email': email,
                'location': attendee.get('location'),
                'displayName': attendee.get('displayName'),
                'phoneNumber': attendee.get('phoneNumber'),
                'comm_id': comm_id
            })
            users_emails.add(email)
            if attendee.get('name'):
                added_names.add(attendee.get('name').lower())

    for speaker in speakers_list:
        
        speaker_name = speaker.get('name', '').strip()
        
        if speaker_name and speaker_name.lower() not in added_names:
            rows.append({
                'user_id': str(uuid.uuid4()),
                'name': speaker_name,
                'email': None,
                'location': None,
                'displayName': None,
                'phoneNumber': None,
                'comm_id': comm_id
            })
            added_names.add(speaker_name.lower())


    # Organisers
    if organiser:
        if organiser not in users_emails:
            rows.append({
                    'user_id': str(uuid.uuid5(uuid.NAMESPACE_DNS, organiser)) if organiser else str(uuid.uuid4()),
                    'name': None,
                    'email': organiser,
                    'location': None,
                    'displayName': None,
                    'phoneNumber': None,
                    'comm_id': comm_id
                })
            users_emails.add(organiser)

    # Hosts
    if host:
        if host not in users_emails:
            rows.append({
                    'user_id': str(uuid.uuid5(uuid.NAMESPACE_DNS, host)) if host else str(uuid.uuid4()),
                    'name': None,
                    'email': host,
                    'location': None,
                    'displayName': None,
                    'phoneNumber': None,
                    'comm_id': comm_id
                })
            users_emails.add(host)
                   
df_user = pd.DataFrame(rows)
df_user.drop_duplicates(inplace=True)

In [20]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_user[['user_id','name','email','location','displayName','phoneNumber']].to_excel(writer,sheet_name='dim_user',index=False)

### bridge_comm_user

In [21]:
comm_user = []

for i in range(len(list_comm_id)):
    comm_id = list_comm_id[i]
    attendees = list_attendees[i] or []
    participants = list_participants[i] or []
    organiser = list_organizer_email[i].lower()
    host = list_host_email[i].lower()
    speakers = list_speakers[i] or []

    attendee_emails = [att.get('email', '').lower() for att in attendees if att.get('email')]
    participant_emails = [p.lower() for p in participants if p]
    speaker_names = [spk.get('name', '').lower() for spk in speakers if spk.get('name')]

    for j in range(len(df_user)):
        
        user_row = df_user.iloc[j]
        
        if user_row['comm_id'] != comm_id:
            continue  # Only process users for this comm_id

        email = user_row['email']
        name = user_row['name']
                
        isAttendee = email.lower() in attendee_emails if email else False
        isParticipant = email.lower() in participant_emails if email else False
        isOrganiser = email.lower() == organiser if email else False
        isHost = email.lower() == host if email else False
        isSpeaker = name.lower() in speaker_names if name else False


        comm_user.append({
            'comm_id': comm_id,
            'user_id': user_row['user_id'],
            'isAttendee': isAttendee,
            'isParticipant': isParticipant,
            'isSpeaker' : isSpeaker,
            'isOrganiser': isOrganiser,
            'isHost' : isHost
        })

df_comm_user = pd.DataFrame(comm_user)
df_comm_user.drop_duplicates(inplace=True)

In [22]:
#Exporting to Excel file:
with pd.ExcelWriter('final_data.xlsx',engine='openpyxl',mode='a') as writer:
    df_comm_user.to_excel(writer,sheet_name='bridge_comm_user',index=False)

## fact_communication

In [23]:
list_comm_id = df['id']
list_raw_id = df['raw_content'].apply(
    lambda x: extract_all_json_objects(x)[0].get('id') if extract_all_json_objects(x) else None)

In [24]:
df.head(2)

,id,comm_type,raw_content,source_id,ingested_at,processed_at,is_processed,subject
0,de819d74-9b7d-4b60-863c-5f7f634a6402,call,"{""id"": ""DDCD444CF2A04CA1BA108EE5"", ""title"": ""U...",32A19D011A4C41F09FE3A2FB,2025-06-12T13:29:09,2025-03-22T11:37:50,True,Profit-focused composite time-frame
1,d4746716-3350-4e7f-952f-b39a8dc430d3,call,"{""id"": ""E462A7000DCA4A688FB880B3"", ""title"": ""E...",485F87C4F6E34F3189D1E3E9,2025-03-11T19:29:43,2025-03-06T01:09:40,True,Ergonomic dedicated process improvement


In [25]:
# fact = []

# for i, comm_id in enumerate(list_comm_id):
#     fact.append({
#         'comm_id' : comm_id,
#         'raw_id' : 
#     })
    
    